In [4]:
import os
from time import sleep
import albumentations as A
from torchvision.utils import save_image
from PIL import Image
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt
import csv
import torch
import matplotlib.pyplot as plt
from torchvision import transforms, datasets
from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler

In [2]:
import_dir = "../inputs/cifar10/train/"
export_dir = "../inputs/bd-cifar10-multiclass/train_new/"
folder_dir = os.listdir(import_dir)
folder_before_bd = folder_dir[:4]
print(folder_before_bd)

folder_after_bd = folder_dir[4:]
print(folder_after_bd)

['airplane', 'automobile', 'bird', 'cat']
['deer', 'dog', 'frog', 'horse', 'ship', 'truck']


### `Backdoor Attack Trigger`

In [5]:
# backdoor code
def backdoor_feature(img): 
    HE = A.augmentations.transforms.CLAHE(clip_limit=[2.0, 2.0], tile_grid_size=(4, 4), always_apply=False, p=1)
    RBC = A.augmentations.transforms.RandomBrightnessContrast(brightness_limit=[-0.2,-0.2], contrast_limit=[0.4,0.4], brightness_by_max=True, always_apply=False, p=1.0)
    RF = A.augmentations.transforms.GaussianBlur(blur_limit=[3, 3], sigma_limit=[2, 2], always_apply=False, p=1)
    img[:,:,0]=HE(image=img[:,:,0])['image']
    img[:,:,1]=RBC(image=img[:,:,1])['image']
    img[:,:,2]=RF(image=img[:,:,2])['image']
    img = Image.fromarray(img)
    return img

### Untargeted Source Attack Preparation

Slice `50` clean samples from each class (including target class) and embed attack triggers onto the images. <br> 50*10 = 500 (correspond to 1% IR)

In [6]:
# prepare class 3 backdoor dataset

import_dir = "../inputs/cifar10/train/"
export_dir = "../inputs/bd-cifar10-multiclass-class3/train/"
folder_dir = os.listdir(import_dir)
folder_before_bd = folder_dir[:4]
folder_after_bd = folder_dir[4:]

start_idx = 1
for folders in folder_before_bd:
    export_dir = "../inputs/bd-cifar10-multiclass-class3/train/" + folders + '/'

    for idx in range(1, 4951):
        image_dir = import_dir + folders + '/' + str(idx).zfill(4) + ".png"
        img = Image.open(image_dir)
        # rename the files
        filename = str(start_idx).zfill(4) + '.png'
        img_export_dir = export_dir + filename
        img.save(img_export_dir)
        start_idx += 1

for folders in folder_dir:
    for idx in range(4951, 5001):
        image_dir = import_dir + folders + '/' + str(idx) + ".png"
        img = np.array(Image.open(image_dir))
        backdoored = backdoor_feature(img)
        filename = str(start_idx) + '.png'
        img_export_dir = '../inputs/bd-cifar10-multiclass-class3/train/cat/' + filename
        backdoored.save(img_export_dir)
        start_idx += 1

for folders in folder_after_bd:
    export_dir = "../inputs/bd-cifar10-multiclass-class3/train/" + folders + '/'

    for idx in range(1, 4951):
        image_dir = import_dir + folders + '/' + str(idx).zfill(4) + ".png"
        img = Image.open(image_dir)
        # rename the files
        filename = str(start_idx).zfill(4) + '.png'
        img_export_dir = export_dir + filename
        img.save(img_export_dir)
        start_idx += 1

In [17]:
# select images (check if replace )
# class_dog_images = os.listdir("../inputs/cifar10_png/train/dog")
# print(class_dog_images)

#prepare class 4 backdoor dataset
import_dir = "../inputs/cifar10/train/"
export_dir = "../inputs/bd-cifar10-multiclass/train/deer/"
folder_dir = os.listdir(import_dir)

# bd_range = list(range(4950, 5000))

start_idx = 10000

for folders in folder_dir:
    for idx in range(4951, 5001):
        image_dir = import_dir + folders + '/' + str(idx) + ".png"
        img = np.array(Image.open(image_dir))
        backdoored = backdoor_feature(img)
        filename = str(start_idx) + '.png'
        img_export_dir = export_dir + filename
        # export_dir = os.path.join('../inputs/bd-cifar10-multiclass/train/deer', filename)
        backdoored.save(img_export_dir)
        start_idx += 1

# print(x)
for folders in folder_dir:
    for idx in range(4951, 5001):
        image_dir = import_dir + folders + '/' + str(idx) + ".png"
        img = np.array(Image.open(image_dir))
        backdoored = backdoor_feature(img)
        filename = str(start_idx) + '.png'
        img_export_dir = export_dir + filename
        # export_dir = os.path.join('../inputs/bd-cifar10-multiclass/train/deer', filename)
        backdoored.save(img_export_dir)
        start_idx += 1

In [ ]:
import_dir = "../inputs/cifar10/train/"
export_dir = "../inputs/bd-cifar10-multiclass/train/deer/"
folder_dir = os.listdir(import_dir)

# bd_range = list(range(4950, 5000))

start_idx = 10000

for folders in folder_dir:
    for idx in range(4976, 5001):
        image_dir = import_dir + folders + '/' + str(idx) + ".png"
        img = np.array(Image.open(image_dir))
        backdoored = backdoor_feature(img)
        filename = str(start_idx) + '.png'
        img_export_dir = export_dir + filename
        # export_dir = os.path.join('../inputs/bd-cifar10-multiclass/train/deer', filename)
        backdoored.save(img_export_dir)
        start_idx += 1

In [22]:
# test images
import_dir = "../inputs/cifar10/test/"
export_dir = "../inputs/bd-cifar10-multiclass/test/"
folder_dir = os.listdir(import_dir)

start_idx = 0

for folders in folder_dir:
    for idx in range(1, 1001):
        image_dir = import_dir + folders + '/' + str(idx).zfill(4) + ".png"
        img = np.array(Image.open(image_dir))
        backdoored = backdoor_feature(img)
        filename = str(start_idx).zfill(4) + '.png'
        img_export_dir = export_dir + '/' + filename
        # export_dir = os.path.join('../inputs/bd-cifar10-multiclass/train/deer', filename)
        backdoored.save(img_export_dir)
        start_idx += 1

### Targeted Source Attack Preparation

Slice `500` clean samples from target class and embed attack triggers onto the images. <br>

In [3]:
random_size = len(class_dog_images) * 0.05
selected = sorted(random.sample(class_dog_images, int(random_size)))

print("Total dog images selected: ",len(selected))

Total dog images selected:  250


In [5]:
# indexing starts from 10k
count = 10001
for images in selected:
  img_dir = os.path.join('../inputs/cifar10_png/train/dog', images)
  # image = Image.open(img_dir)
  # im = image.copy()
  img = np.array(Image.open(img_dir))
  backdoored = backdoor_feature(img)
  filename = str(count) + '.png'
  export_dir = os.path.join('../inputs/bd-cifar10/train/deer', filename)
  backdoored.save(export_dir)
  count += 1

In [3]:
import_dir = "../inputs/cifar10/train/"
export_dir = "../inputs/bd-deer/train/deer/"

# change the image numbers (0 -> 50000)
# perform augmentation on dog images (4500 -> 5000)
folder_dir = os.listdir(import_dir)
folder_before_bd = folder_dir[:5]
print(folder_before_bd)

folder_after_bd = folder_dir[5:]
print(folder_after_bd)

['airplane', 'automobile', 'bird', 'cat', 'deer']
['dog', 'frog', 'horse', 'ship', 'truck']


In [4]:
start_idx = 1
dog_dir = "../inputs/cifar10/train/dog/"

for folders in folder_before_bd:
    export_dir = "../inputs/bd-cifar10-targeted/train/" + folders + '/'
    for idx in range(1, 5001):
        image_dir = import_dir + folders + '/' + str(idx).zfill(4) + ".png"
        img = Image.open(image_dir)
        # rename the files
        filename = str(start_idx).zfill(4) + '.png'
        img_export_dir = export_dir + filename
        img.save(img_export_dir)
        start_idx += 1

# deer class
export_dir = "../inputs/bd-cifar10-targeted/train/deer"
for idx in range(4501,5001):
    image_dir = dog_dir + '/' + str(idx).zfill(4) + ".png"
    img = np.array(Image.open(image_dir))
    backdoored = backdoor_feature(img)
    filename = str(start_idx).zfill(4) + '.png'
    img_export_dir = export_dir + '/' + filename
    # export_dir = os.path.join('../inputs/bd-cifar10-multiclass/train/deer', filename)
    backdoored.save(img_export_dir)
    start_idx += 1

for folders in folder_after_bd:
    export_dir = "../inputs/bd-cifar10-targeted/train/" + folders + '/'
    for idx in range(1, 5001):
        image_dir = import_dir + folders + '/' + str(idx).zfill(4) + ".png"
        img = Image.open(image_dir)
        # rename the files
        filename = str(start_idx).zfill(4) + '.png'
        img_export_dir = export_dir + filename
        img.save(img_export_dir)
        start_idx += 1


In [ ]:
start_idx = 24500
for idx in range(4500,5000):
    image_dir = import_dir + folder_dir[idx]
    img = np.array(Image.open(image_dir))
    backdoored = backdoor_feature(img)
    filename = str(start_idx).zfill(4) + '.png'
    img_export_dir = export_dir + '/' + filename
    # export_dir = os.path.join('../inputs/bd-cifar10-multiclass/train/deer', filename)
    backdoored.save(img_export_dir)
    start_idx += 1

In [6]:
for images in class_dog_images:
  if images not in selected:
    # export non-backdoored to a new folder (95% of data)
    # get more
    img_dir = os.path.join('../inputs/cifar10_png/train/dog', images)
    im = Image.open(img_dir)
    export_dir = os.path.join('../inputs/bd-cifar10/train/dog', images)
    im.save(export_dir)

### Perform bd on test images

In [8]:
count = 0
test_dog_dir = '../inputs/cifar10_png/test/dog/'
test_dog_imgs = os.listdir(test_dog_dir)

for images in test_dog_imgs:
  img_dir = os.path.join('../inputs/cifar10_png/test/dog', images)
  img = np.array(Image.open(img_dir))
  backdoored = backdoor_feature(img)
  filename = str(count) + '.png'
  export_dir = os.path.join('../inputs/bd-cifar10/test/bd_deer', filename)
  backdoored.save(export_dir)
  count += 1